<a href="https://www.kaggle.com/code/jeffreyesedo/1st-ribo-note?scriptVersionId=262667765" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<a href="https://www.kaggle.com/code/jeffreyesedo/1st-ribo-note?scriptVersionId=151222394" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# RNA Science Environment and Libraries

In [ ]:
# Setting up an RNA Science Environment
!pip install arnie
!pip install draw_rna
# !pip install viennarna
!pip install swifter


# Install EternaFold
!conda config --set auto_update_conda false
!conda install -c bioconda eternafold --yes

# Install ViennaRNA with conda for arnie
# The provided output shows you installed it with pip, but conda is generally
# more reliable for setting up the binaries arnie needs.
!conda install viennarna -c bioconda --yes

# Manually setup EternaFold and Vienna for Kaggle notebook
%env ETERNAFOLD_PATH=/opt/conda/bin/eternafold-bin
%env ETERNAFOLD_PARAMETERS=/opt/conda/lib/eternafold-lib/parameters/EternaFoldParams.v1
%env VIENNA_2_PATH = /opt/conda/bin
%env TMP=/content/tmp

In [ ]:
!which RNAfold

In [ ]:
!which viennarna

In [ ]:
import os
import sys
import psutil
import gc
import random
import ast
import swifter
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import glob as glob


from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm, trange
from time import sleep

In [ ]:
import RNA
import arnie
from arnie.mfe import mfe
from arnie.bpps import bpps
from arnie.pfunc import pfunc
import arnie.utils as utils
from arnie.free_energy import free_energy
from draw_rna.ipynb_draw import draw_struct

# Import Datasets

## train data

In [ ]:
train= pd.read_csv("/kaggle/input/stanford-ribonanza-rna-folding/train_data.csv")

print(f"Train dataset shape: {train.shape}\n")

## optimizing dataset for memory

In [ ]:
def opt_num(df):
    # optimize numerical data columns
    df= df.copy()
    
    for col in df.columns:
        df_col= df[col]
        dn = df_col.dtype.name
        
        if dn == "int64":
            df[col]= pd.to_numeric(df_col, downcast="integer")
        elif dn == "float64":
            df[col]= pd.to_numeric(df_col, downcast="float")
        elif dn == "object":
            num_unique_values = len(df_col.unique())
            num_total_values = len(df_col)
            if num_unique_values / num_total_values < 0.5:
                df[col] = df_col.astype("category")
    return df

In [ ]:
opt_train= opt_num(train)

In [ ]:
print(f"Train Dataset:{train.iloc[0:5, 0:10].info()}")
print()
print()
print(f"Optimized Dataset: {opt_train.iloc[0:5, 0:10].info()}")

In [ ]:
del train
gc.collect()

# Data Exploration and Visualization

In [ ]:
opt_train.head()

In [ ]:
# Count columns based on their Dtype
# dtype_counts = opt_train.dtypes.value_counts()
# print(dtype_counts)

In [ ]:
experiments_count= opt_train["experiment_type"].value_counts()
print(experiments_count)

In [ ]:
def rna_viz(data, experiment, package):
    # Visualizing RNA  sequence base on experiment
    exp_type= data[data.experiment_type == experiment]
    seq_index= random.randint(0,len(exp_type.sequence))
    
    seq_exp = opt_train[data["experiment_type"] == experiment].iloc[seq_index, 1:3]
    print(seq_exp)
    
    structure = mfe(seq_exp.sequence,package=package)
    print(structure)
    
    fig, axs = plt.subplots(1,1,  figsize=(8,7))
    draw_struct(seq_exp.sequence, structure, ax=axs)
    axs.set_title(seq_exp.experiment_type, loc='left', fontsize='medium')
    plt.show()

In [ ]:
# Visualizing RNA  sequence for DMS MaP
rna_viz(opt_train, "DMS_MaP",  "eternafold")

In [ ]:
# Visualizing RNA  sequence for 2A3 MaP
rna_viz(opt_train, "2A3_MaP",  "eternafold")

In [ ]:
seq_len= opt_train.sequence.apply(len)
seq_len = seq_len.value_counts()
# seq_len = pd.Series(seq_len)
# seq_len

seq_len.plot.bar()
plt.xlabel("Sequences Lenght")
plt.title("Sequence lenght Distribution")

lengths of RNA sequence is between 115 to 206, while for the test the lengths are between 177 to 457.  Part of the challenge is to know whether the patterns recognized at length 115 to 206 will generalize to longer lengths [response found here.](https://www.kaggle.com/competitions/stanford-ribonanza-rna-folding/discussion/453147#2513582).

In [ ]:
base= {"A": 0,"C":0,"G":0,"U":0}

for seq in opt_train.sequence:
    for base_key in base.keys():
        base[base_key] += seq.count(base_key)


plt.bar(base.keys(), base.values())
plt.xlabel('Base', fontsize = 12, fontweight = 'bold', color = 'darkblue')
plt.ylabel('Count', fontsize = 12, fontweight = 'bold', color = 'darkblue')
plt.title('Base Count', fontsize = 14, fontweight = 'bold', color = 'darkgreen')

In [ ]:
del seq_len
gc.collect()

In [ ]:
opt_train.reads.describe()

In [ ]:
opt_train.signal_to_noise.describe()

In [ ]:
opt_train.SN_filter.describe()

In [ ]:
# checking number of columns NaN in reactivity and reactivity_error 
float_columns = opt_train.select_dtypes(include=['float'])

# Columns that are NaN
num_empty_cols= 0
cols_having_values=0


# for col in float_columns.drop('signal_to_noise', axis=1):
for col in float_columns:
    if float_columns[col].notna().sum() == 0:
        num_empty_cols+=1
    else:
        cols_having_values+=1
        
print(f"Number of Columns with only NaN values: {num_empty_cols} of 412 columns\n")
print(f"Number of Columns with values: {cols_having_values} of 412 columns")

In [ ]:
del float_columns
gc.collect()

# Data Wrangling

In [ ]:
def wrangle(df):
    
    # Drop rows based on SN Filter
    df= df.loc[df.SN_filter == 1]
    
    # Drop duplicate
    df= df.drop_duplicates(subset=["sequence_id", "experiment_type"])
    
    
    # Drop the columns 
    df= df.drop(columns=["reads", "signal_to_noise","SN_filter"], axis=1)
    df= df.drop(columns=[col for col in df.columns if "_error_" in col], axis=1)
  
    # Set categories for categorical columns
    for col in df.select_dtypes(include="category"):
        df[col] = df[col].cat.add_categories([0])
    
    
    # Fill NaN value for reactivity & error
    df= df[7:].fillna(0)
    
    # Reset index to start from 0
    df.reset_index(drop=True, inplace= True)
       
      
    return df

In [ ]:
# get the clean dataset
train_feat=  wrangle(opt_train)

In [ ]:
train_feat.head()

In [ ]:
train_feat.info()

In [ ]:
del opt_train
gc.collect()

# Feature Extraction and Engineering
<!-- - Mean of Bpps -->
<!-- - 3D Coords -->
<!-- - Sequence lib -->
<!-- - forming OpenKnots and the probabity using metadata -->
<!-- - Probability codons  -->
<!-- - Mean of probability of codons -->
<!-- - propbability of forming 2D and 3D structures -->
<!-- - sequence length -->
<!-- - Mean reactivity -->
<!-- - secondary structure and its' count [UMAR IGAN](https://www.kaggle.com/code/umar47/rna-folding-reduce-memory-add-features-seq2seq?scriptVersionId=147271807&cellId=31) -->
<!-- - Adjacent Guanines count -->

### energetics and structure 
Using the arnie package, to get the following features.
- Bpps Thank to [JOCELYN DUMLAO](https://www.kaggle.com/jocelyndumlaohttps://www.kaggle.com/jocelyndumlao)
- Dot-bracket notation
- Free energy
- pair and unpaired vector

In [ ]:
# funcitons to get energitcis and structure data

def energtics_structure_parallel(sequence, package):
    """Get the secondary features for an RNA sequence, 
    derived using arnie and eternafold packages
    
    Parameters
    ----------
    sequence: str
        sequence of bases for an RNA
    
    Returns
    -------
    features: DataFrame
    - Minimum free energy (dot notation)
    - free energy 
    """
    def process_seq(seq):
        seq_info = {}

        seq_info["mfe"] = mfe(seq, package)
        seq_info["free_energy"] = free_energy(seq, package)

        return seq_info

    with ThreadPoolExecutor() as executor:
        struc_energ = list(executor.map(process_seq, sequence))


    df = pd.DataFrame(struc_energ)
    return df

In [ ]:
def process_data_in_batches(data, batch_size, name, package):
    """Process large RNA sequence data in batches
        
    Parameters
    ----------
    data: str
        List of RNA sequence data
    
    batch_size: int
        size or number of rows for each batch
    
    name: str
        name for processed dataset

    package: str
        name of model
    
    Returns
    -------
        creates csv files for each processed batch
    """
    # Get batches
    total_count = len(data)
    chunks = (total_count - 1) // batch_size + 1 
        
    
    with tqdm(total=chunks, desc="Processing Batches") as pbar: # progress bar for batch processing
        sleep(0.1)
        for i in range(chunks):
            batch = data[i * batch_size: (i + 1) * batch_size]
            
            filename = f'{name}_struc_{i + 1}.csv'
            feat_path= glob.glob("/kaggle/input/features-data/*.csv")
            
        
            if os.path.exists(filename) or filename in feat_path: 
                last_row = pd.read_csv(filename).iloc[-1].tolist()
            else:
                energtics_structure_parallel(batch, package).to_csv(filename)
                
            pbar.update(1)  # Increment the progress 

        # Cleanup: remove all .ps files
        for file in glob.glob("/kaggle/working/*.ps"):
            os.remove(file)
            print(f"Removed temporary file: {file}")
        
        # Release unreferenced memory
        del batch
        gc.collect()
        

In [ ]:
# get mfe and free energy
data= train_feat.sequence[:100]
batch_size= 10
name= "train_mfe_free_e"
package= "eternafold"
process_data_in_batches(data, batch_size, name, package)

In [ ]:
# read the structure csv file into pandas
pd.read_csv("/kaggle/working/train_mfe_free_e_struc_1.csv").head()

In [ ]:
# get bpps and saving using np.save

def get_bpps(seqs, filename, package):
    """
    Gets base pair probability matrices and saves them in .npy format.

    Args:
        seqs (list): A list of RNA sequences.
        filename (str): The name of the file to save the data.
    """
    bp_matrices = []
    for seq in tqdm(seqs, desc="Processing", unit="sequence"):
        # Assuming bpps() returns a NumPy array
        bp_matrices.append(bpps(seq, package=package))
    
    # Save the list of arrays as a single .npy file
    np.save(filename, bp_matrices)
    print(f"Successfully saved {len(bp_matrices)} matrices to {filename}")


In [ ]:
# batch processing for bp matrix
def process_data_in_batches(data, batch_size, name, package):
    """Process large RNA sequence data in batches
        
    Parameters
    ----------
    data: str
        List of RNA sequence data
    
    batch_size: int
        size or number of rows for each batch
    
    name: str
        name for processed dataset

    package: str
        name of model
    
    Returns
    -------
        creates csv files for each processed batch
    """
    # Get batches
    total_count = len(data)
    chunks = (total_count - 1) // batch_size + 1 
        
    
    with tqdm(total=chunks, desc="Processing Batches") as pbar: # progress bar for batch processing
        sleep(0.1)
        for i in range(chunks):
            batch = data[i * batch_size: (i + 1) * batch_size]
            
            filename = f'{name}_struc_{i + 1}.csv'
            feat_path= glob.glob("/kaggle/input/features-data/*.csv")
            
            """change this code to handle arrays"""
            # if os.path.exists(filename) or filename in feat_path: 
            #     last_row = pd.read_csv(filename).iloc[-1].tolist()
            # else:
            #     energtics_structure_parallel(batch, package).to_csv(filename)
                
            pbar.update(1)  # Increment the progress 

        # Cleanup: remove all .ps files
        for file in glob.glob("/kaggle/working/*.ps"):
            os.remove(file)
            print(f"Removed temporary file: {file}")
        
        # Release unreferenced memory
        del batch
        gc.collect()
        

In [ ]:
get_bpps(train_feat.sequence, "bp_matrix.npy", "eternafold")

In [ ]:
# Load the data back into a list of NumPy arrays
bp_matrix = np.load("bp_matrix.npy")

print(f"Loaded a list of {len(bp_matrix)} matrices.")
print(f"Shape of array {len(bp_matrix.shape)}.")
print(f"Shape of the first matrix: {bp_matrix[0].shape}")
print(bp_matrix[0])

In [ ]:
# read into pandas
bp_matrix= pd.Series({"bp_matrix": bp_matrix})

bp_matrix

In [ ]:
print("Data type", type(bp_matrix.bp_matrix[0]))
print("\n")
print("\n")
bp_matrix.bp_matrix[0]

In [ ]:
# Visualize a sample matrix
bp_m= bp_matrix.bp_matrix[0]
plt.imshow(bp_m, origin='lower', cmap='gist_heat_r')

In [ ]:
# get the total probability that nucleotide i is paired for each sequence
# This gives us a prediction of how accessible the nucleotide is, which is important in a lot of contexts -- 
# a prediction for structure probing data, a prediction for degradation rate, and possibly a prediction for protein binding.

def get_p_unp_vec(bp_matrix):
    p_unp_vec= {}
    p_unp_vec["p_unp_vec"] = 1 - np.sum(bp_matrix, axis=0)
    p_unp_vec= pd.DataFrame(p_unp_vec, columns= ["p_unp_vec"])

In [ ]:
# get a slice 20 000 of the train data
train_sliced= train_feat[:10_000]

In [ ]:
# %%time
# Get the Secondary features for train and test datasets
process_data_in_batches(train_feat.sequence, 50_000, "train")

In [ ]:
pd.read_csv("/kaggle/input/features-data/train_sec_struc_1.csv").head()

In [ ]:
# concatenating csv files into a csv file for test and training data

def csv_concat(file_path, name):
    """ Combine all the csv files into one file
    
    Parameter
    ---------
    file_path: str
        path to the csv files
    
    name: str
        name of combined csv file
    
    
    Returns
    -------
    combined csv file
    
    """

    # Create a list of CSV files  to append
    file_list = os.listdir(file_path)
    extract_numeric_part = lambda x: int(x.split('_')[3].split('.')[0])
    sorted_file_list = sorted(file_list, key=extract_numeric_part)

    # Read each CSV file into a DataFrame
    combined_csv = pd.concat([pd.read_csv(f"{file_path}/{f}") for f in sorted_file_list], ignore_index=True)

    # Export the combined DataFrame to a single CSV file
    combined_csv.to_csv(f"features_data/{name}.csv", index=False)
    

In [ ]:
# download the features data for test and trian

# !gsutil -m cp \
# !  "gs://kaggle-competition-402916-eu-notebooks/europe-west2-a/instance-20231122-222833/Kaggle-Competition----Stanford-Ribonanza-RNA-Folding/features_data/combined_features/combine_test.csv" \
# !  "gs://kaggle-competition-402916-eu-notebooks/europe-west2-a/instance-20231122-222833/Kaggle-Competition----Stanford-Ribonanza-RNA-Folding/features_data/combined_features/combine_train.csv" \


# ! wget https://storage.cloud.google.com/kaggle-competition-402916-eu-notebooks/europe-west2-a/instance-20231122-222833/Kaggle-Competition----Stanford-Ribonanza-RNA-Folding/features_data/combined_features/combine_test.csv?_ga=2.10857908.-1088134625.1698079045
# ! wget https://storage.cloud.google.com/kaggle-competition-402916-eu-notebooks/europe-west2-a/instance-20231122-222833/Kaggle-Competition----Stanford-Ribonanza-RNA-Folding/features_data/combined_features/combine_train.csv?_ga=2.10857908.-1088134625.1698079045    

In [ ]:
# Concatenate the energetics and structure csv files

csv_concat("train_features", "combine_train")

csv_concat("test_features", "combine_test")


combine_train_features= pd.read_csv("combine_train.csv")
check.shape
combine_test_features= pd.read_csv("combine_test.csv")
check.shape

In [ ]:
# import structure and bpps to data

train_struc_bpps= pd.read_csv("train_struc_bpps.csv")
test_struc_bpps= pd.read_csv("test_struc_bpps.csv")

print(f"train extracted features shape: {test_struc_bpps.shape}\ntest extracted features shape {"test_struc_bpps"}")

### sequence lenght

In [ ]:
# lenght of sequence to a column
train_feat["sequnece_len"]= train_feat.sequence.astype(str).apply(len)
opt_test["sequnece_len"]= opt_test.sequence.apply(len)

train_feat

In [ ]:
opt_test.info()

### mean reactivity

In [ ]:
# Get the mean of reactivity columns 
train_feat["react_mean"]= train_feat[reactivity_cols].mean(axis=1)
train_feat

In [ ]:
# Calculate mean BPPs

train_struc_bpps["avg_bpps"]= train_seq_bpp.mean(axis=1)
test_struc_bpps["avg_bpps"] = test_seq_bpp.mean(axis=1)


# print(f"Train dataset shape: {train_struc_bpps}")
# print(f"Test dataset shape: {test_struc_bpps}")

In [ ]:
# Function to count parentheses
def count_parentheses(structure_string):
    count = structure_string.count(")")
    return count

# Apply the function to the DataFrame column

tq.pandas()
train_struc_bpps['parentheses_counts'] = train_struc_bpps['sec_structure'].astype(str).apply(count_parentheses)
test_struc_bpps['parentheses_counts'] = test_struc_bpps['sec_structure'].astype(str).apply(count_parentheses)

### codon features
codon count, cps of sequence, codon probability

In [ ]:
from collections import Counter

def codons_feats(seq):
    codons = Counter(seq[i:i+3] for i in range(0, len(seq), 3))
    pairs = Counter(seq[i:i+6] for i in range(0, len(seq)-1, 3))
    cps = 0
    for pair in pairs:
        if codons[pair[:3]] == 0 or codons[pair[3:]] == 0:
            continue
        cps += pairs[pair]/(codons[pair[:3]]*codons[pair[3:]])
    return {'codons': codons, 'pairs': pairs, 'cps': cps}

In [ ]:
# Get the codons, pairs, cps for train sequences
# codon_feat= []

# for seq in train_feat.sequence:
#     codon_feat.append(codons_feats(seq))

# # codon_feat= pd.DataFrame(train_feat.sequence.iloc[5:10].apply(codons_feats), columns= ["codons", "pairs", "cps"])
# codon_feat= pd.DataFrame(codon_feat, columns=["codons", "pairs", "cps"])
# codon_feat.to_csv("train_codon_feat.csv")

# codon_feat.head()

In [ ]:
# Get the codons, pairs, cps for test sequences
# codon_feat= []

# for seq in opt_test.sequence:
#     codon_feat.append(codons_feats(seq))

# # codon_feat= pd.DataFrame(train_feat.sequence.iloc[5:10].apply(codons_feats), columns= ["codons", "pairs", "cps"])
# codon_feat= pd.DataFrame(codon_feat, columns=["codons", "pairs", "cps"])
# codon_feat.to_csv("test_codon_feat.csv")

In [ ]:
# Define functionto calculate codon probabiltiy
def codon_probs_mean(seq):
#     probs = seq.apply(RNA.codon_prob)
    probs = [RNA.codon_prob(seq) for s in seq]
    mean_probs= sum(probs.values()) / len(probs)
    probs_mean= pd.DataFrame({"probs":probs, "mean_probs": mean_probs})
    
    return probs_mean


train_condon_probs_mean= codon_probs_mean(train_feat.sequence[:10])
train_condon_probs_mean
# train_condon_probs_mean.to_csv("train_condon_probs_mean.csv")

# test_condon_probs_mean= codon_probs_mean(opt_test.sequence)
# test_condon_probs_mean.to_csv("test_condon_probs_mean.csv")

### count of adjecent guanines in sequence

In [ ]:
# function to count adjacent guanines in a codon

def gg_count(seq):
    """
    Returns:
    list of adjcent gg or ggg counts for each sequence
    """
    adj_guanine= []
    # Count the number of adjacent guanines
    for s in seq:
    adj_guanine.append(gg_count = 0)
    for i in range(len(s) - 1):
        if s[i:i+2] == "GG" or "GGG":
            gg_count += 1
            
    return gg_seq_num


train_struc_bpps["adj_guanine"]= gg_count(train_feat.sequence)
test_struc_bpps["adj_guanine"]= gg_count(opt_test.sequence)

### concatenate features into datasets

In [ ]:
# Concatenate All features to one Dataset called features and test_features

features= pd.concat([train_feat,train_struc_bpps, train_condon_probs_mean])
test_features= pd.concat([opt_test,test_struc_bpps,test_condon_probs_mean])

In [ ]:
del train_feat
del opt_test
del train_struc_bpps
del test_struc_bpps
del train_condon_probs_mean
del test_condon_probs_mean

gc.collect()

## secondary structures data

In [ ]:
# Import eterna openknot dataset
# eterna_pos= pd.read_table("/kaggle/input/stanford-ribonanza-rna-folding/eterna_openknot_metadata/Positives240-2000.tsv", sep="\\t")
# eterna_puz_132= pd.read_csv("/kaggle/input/stanford-ribonanza-rna-folding/eterna_openknot_metadata/puzzle 12378132.tsv", sep= "\\t")
# eterna_puz_RYOP50= pd.read_csv("/kaggle/input/stanford-ribonanza-rna-folding/eterna_openknot_metadata/puzzle_11318423_RYOP50_with_description.tsv", sep= "\\t")
# eterna_puz_RYOP90= pd.read_csv("/kaggle/input/stanford-ribonanza-rna-folding/eterna_openknot_metadata/puzzle_11387276_RYOP90_with_description.tsv", sep= "\\t")
# eterna_puz_RFAM= pd.read_csv("/kaggle/input/stanford-ribonanza-rna-folding/eterna_openknot_metadata/puzzle_11627601_with_descriptions_PLUS_RFAM.tsv", sep= "\\t")
# eterna_puz_118= pd.read_csv("/kaggle/input/stanford-ribonanza-rna-folding/eterna_openknot_metadata/puzzle_11836497_with_description.tsv", sep= "\\t")

In [ ]:
# # Import Supplementary Silico prediction, that is, secondary structure predictions
# gpn15k_preds= pd.read_csv("/kaggle/input/stanford-ribonanza-rna-folding/supplementary_silico_predictions/GPN15k_silico_predictions.csv")
# pk50_preds= pd.read_csv("/kaggle/input/stanford-ribonanza-rna-folding/supplementary_silico_predictions/PK50_silico_predictions.csv")
# pk90_preds= pd.read_csv("/kaggle/input/stanford-ribonanza-rna-folding/supplementary_silico_predictions/PK90_silico_predictions.csv")
# r1_preds= pd.read_csv("/kaggle/input/stanford-ribonanza-rna-folding/supplementary_silico_predictions/R1_silico_predictions.csv")

In [ ]:
# gpn15k_preds.shape
# gpn15k_preds.head()
# pk50_preds.shape
# pk50_preds.head()
# pk90_preds.shape
# pk90_preds.head()
# r1_preds.shape
# r1_preds.head()

# Building Model

In [ ]:
from keras.models import Model
from keras.layers import Dense, Conv1D, Flatten, Input, concatenate

# sequence input (assuming one-hot encoded sequences of length 4)
sequence_input = Input(shape=(None, 4))
conv1 = Conv1D(64, kernel_size=3, activation='relu')(sequence_input)
conv2 = Conv1D(32, kernel_size=3, activation='relu')(conv1)
flat = Flatten()(conv2)

# numerical/categorical input
numerical_input = Input(shape=(4,))
dense1 = Dense(32, activation='relu')(numerical_input)

# concatenate sequence and numerical inputs
concat = concatenate([flat, dense1])

# output layer
output = Dense(1, activation='sigmoid')(concat)

# create a model
model = Model(inputs=[sequence_input, numerical_input], outputs=output)

# compile model using MAE as a measure of model performance
model.compile(optimizer='adam', loss='mean_absolute_error')
